In [3]:
import multiprocessing as mp
from pathlib import Path
import os
import json
import pandas as pd
from tqdm import tqdm
from lhotse import CutSet, RecordingSet, SupervisionSet, MonoCut
from lhotse.cut import append_cuts
from mylhotse.voxpopuli import prepare_voxpopuli
import logging
from functools import reduce
from datasets import load_dataset, DownloadConfig
logging.basicConfig(level=logging.INFO)

In [4]:
# directory paths to save audio and transcript files
IN_DIR = "../datasets/LongSpeechSource/voxpopuli"
# directory paths to save metadata and processed aduio files
OUT_DIR = '../datasets/LongSpeech'

In [5]:
config = json.load(open(os.path.join(OUT_DIR, 'metadata.json')))
AVG_DURATION = config['avg_duration']
SAMPLE_RATE = config['sample_rate']
OUT_FILE_NAME = config['source']
prev_amount = config['amount']
print(prev_amount)
task = "asr"
lang = "en"

8411


In [6]:
OUT_FILE_NAME = "mock_out.jsonl"

In [8]:
manifests = prepare_voxpopuli(corpus_dir=IN_DIR, output_dir=OUT_DIR, task=task, lang=lang, num_jobs=15)

INFO:root:Preparing recordings (this may take a few minutes)...
Scanning audio files (*.ogg): 9572it [00:03, 3174.26it/s] 
INFO:root:Using pre-downloaded annotations ../datasets/LongSpeech/asr_en.tsv.gz
100%|██████████| 412484/412484 [00:01<00:00, 217766.37it/s]


In [10]:
rs = manifests['test']['recordings']
ss = manifests['test']['supervisions']
cuts = CutSet.from_manifests(recordings=rs, supervisions=ss)

In [8]:
"""
cuts = CutSet()
for part in manifests.keys():
    rs = manifests[part]['recordings']
    ss = manifests[part]['supervisions']
    cut = CutSet.from_manifests(recordings=rs, supervisions=ss)
    cuts += cut
"""

In [11]:
spilted_cuts = cuts.trim_to_supervision_groups(max_pause=5).filter(lambda cut: cut.duration > 10).sort_by_recording_id().merge_supervisions(merge_policy="keep_first")

In [12]:
def pack_cuts_to_long_audio(
    cuts: CutSet,
    target_duration: float = 600.0,
    starting_id: int = 0,
) -> (CutSet, int):
    final_long_cuts = []
    buffer_cut = None

    for cut in cuts:
        buffer_cut = cut if buffer_cut is None else buffer_cut.append(cut)
        if buffer_cut.duration >= target_duration:
            final_long_cuts.append(buffer_cut.with_id(f"{starting_id:06d}"))
            starting_id += 1
            buffer_cut = None

    return CutSet.from_cuts(final_long_cuts), starting_id

In [13]:
grouped_cuts, new_amount = pack_cuts_to_long_audio(spilted_cuts, target_duration=600.0, starting_id = prev_amount)
grouped_cuts.to_jsonl(OUT_DIR + "/grouped_cuts.jsonl")

In [17]:
def json_from_voxpopuli_to_allaudios(one_cut, lang = "en"):
    """
    Convert a single LibriSpeech json record to a list of LongSpeech metadata.
    """
    sources = []
    speakers = set()
    total_dur = 0
    transcripts = []
    slices = []

    for subcut in one_cut["tracks"]:
        total_dur += subcut["cut"]["duration"]
        full_pth = subcut["cut"]["recording"]["sources"][0]["source"]
        slices.append([subcut["cut"]["start"], subcut["cut"]["duration"]])
        sources.append(full_pth.split("voxpopuli")[-1])
        [speakers.add(s["speaker"]) for s in subcut["cut"]["supervisions"] if s["speaker"]]
        transcript_param = " ".join([s["text"] for s in subcut["cut"]["supervisions"] if s["text"]])
        if transcript_param != "":
            transcripts.append(transcript_param)
        else:
            print(subcut)

    return {
        "id": one_cut["id"],
        "source_ds": "voxpopuli",
        "duration_sec": total_dur,
        "audio_auto": False,
        "text_auto": False,
        "language": lang,
        "num_speakers": len(speakers),
        "num_switches": len(one_cut["tracks"]),
        "slice": slices,
        "transcribe": " ".join(transcripts),
        "components": sources,
    }

In [18]:
def convert_record(source_jsonl_path: str, target_jsonl_path: str, map_fn, lang: str):
    with open(source_jsonl_path, "r", encoding="utf-8") as src_f, \
         open(target_jsonl_path, "a", encoding="utf-8") as tgt_f:
        for line in src_f:
            item = json.loads(line)
            new_item = map_fn(item, lang)
            tgt_f.write(json.dumps(new_item, ensure_ascii=False) + "\n")

In [19]:
convert_record(os.path.join(OUT_DIR, "grouped_cuts.jsonl"),
               os.path.join(OUT_DIR, OUT_FILE_NAME),
               json_from_voxpopuli_to_allaudios, lang)

In [20]:
def save_audios_from_cutset(cutset, out_dir, num_jobs=1):
    """
    Save audios from a CutSet to the specified directory.
    """
    for cut in tqdm(cutset):
        cut.save_audio(os.path.join(out_dir, f"{cut.id}.wav"))
save_audios_from_cutset(grouped_cuts, os.path.join(OUT_DIR, 'mock_out'))

100%|██████████| 26/26 [00:32<00:00,  1.26s/it]
